# Tarea 4 - Punto 1: Reactor PFR con Elementos Finitos

Solución numérica del estado estacionario de concentración en reactor PFR mediante:
1. Método de Galerkin con polinomios de Legendre (no discretizado)
2. Método de elementos finitos con polinomios de Lagrange (discretizado)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial.legendre import Legendre, leggauss
import os

os.makedirs('figuras', exist_ok=True)

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 1. Parámetros del problema y solución analítica

In [ ]:
# Parámetros del reactor PFR
D = 5000.0      # [m²/hr] - difusión/dispersión
U = 100.0       # [m/hr] - velocidad advectiva
k = 2.0         # [1/hr] - tasa de reacción
L = 100.0       # [m] - longitud del reactor
c_inlet = 100.0 # [mol/L] - concentración de entrada

def solucion_analitica(x, D, U, k, L, c_inlet):
    """
    Solución analítica de la EC diferencial del reactor PFR.
    Ref: Bird, Stewart & Lightfoot (2002)
    """
    raiz = np.sqrt(1 + 4*k*D/U**2)
    lambda1 = (U/(2*D)) * (1 + raiz)
    lambda2 = (U/(2*D)) * (1 - raiz)
    denom = (U - D*lambda1) * lambda2 * np.exp(lambda2*L) - (U - D*lambda2) * lambda1 * np.exp(lambda1*L)
    c = (U * c_inlet / denom) * (lambda2 * np.exp(lambda2*L) * np.exp(lambda1*x) - 
                                   lambda1 * np.exp(lambda1*L) * np.exp(lambda2*x))
    return c

def gauss_legendre_integral(f, a, b, n=80):
    """Integración numérica mediante cuadratura de Gauss-Legendre."""
    xi, wi = leggauss(n)
    x_mapped = 0.5 * (b - a) * xi + 0.5 * (b + a)
    w_mapped = 0.5 * (b - a) * wi
    return np.dot(w_mapped, f(x_mapped))

## 2. Implementación del método no discretizado

Formulación de Galerkin con funciones base de Legendre.

In [ ]:
def Nm(xi, m):
    """Polinomio de Legendre de orden m."""
    return Legendre.basis(m)(xi)

def dNm(xi, m):
    """Derivada del polinomio de Legendre."""
    return Legendre.basis(m).deriv()(xi)

def Ke(i, j, D, U, k, L, quad_n=80):
    """
    Elemento de la matriz de rigidez K_ij.
    Incluye términos difusivo, advectivo y reactivo.
    """
    # Término difusivo
    integrand1 = lambda xi: dNm(xi, i) * dNm(xi, j)
    I1 = gauss_legendre_integral(integrand1, -1, 1, n=quad_n)
    term1 = D * (2.0/L) * I1
    
    # Término advectivo
    integrand2 = lambda xi: Nm(xi, i) * dNm(xi, j)
    I2 = gauss_legendre_integral(integrand2, -1, 1, n=quad_n)
    term2 = U * I2
    
    # Término reactivo
    integrand3 = lambda xi: Nm(xi, i) * Nm(xi, j)
    I3 = gauss_legendre_integral(integrand3, -1, 1, n=quad_n)
    term3 = k * (L/2.0) * I3
    
    return term1 + term2 + term3

def Fe(i, U, c_inlet):
    """Vector de carga considerando BC de entrada."""
    return U * c_inlet * Nm(-1.0, i)

def assemble_KF_legendre(M, D, U, k, L, c_inlet, quad_n=80):
    """Ensamblaje de matrices globales K y F."""
    K = np.zeros((M, M))
    F = np.zeros(M)
    for i in range(M):
        for j in range(M):
            K[i, j] = Ke(i, j, D, U, k, L, quad_n=quad_n)
        F[i] = Fe(i, U, c_inlet)
    
    # Condición de contorno tipo Robin en x=0
    for i in range(M):
        for j in range(M):
            K[i, j] += U * Nm(-1.0, i) * Nm(-1.0, j)
    
    return K, F

## 3. Implementación del método discretizado (FEM)

Elementos finitos con funciones de forma de Lagrange lineales.

In [ ]:
def create_mesh(L, n_elem):
    """Genera malla uniforme de elementos finitos."""
    nodes = np.linspace(0, L, n_elem + 1)
    h = L / n_elem
    connectivity = np.zeros((n_elem, 2), dtype=int)
    for e in range(n_elem):
        connectivity[e, :] = [e, e+1]
    return nodes, h, connectivity

def N_lagrange(xi, i):
    """Funciones de forma lineales de Lagrange."""
    return 1 - xi if i == 0 else xi

def dN_lagrange(i):
    """Derivadas de las funciones de forma."""
    return -1.0 if i == 0 else 1.0

def Ke_local(h_e, D, U, k, quad_n=10):
    """Matriz de rigidez local 2x2 para elemento lineal."""
    K_local = np.zeros((2, 2))
    xi_gauss, w_gauss = leggauss(quad_n)
    xi_gauss = 0.5 * (xi_gauss + 1)
    w_gauss = 0.5 * w_gauss
    
    for i in range(2):
        for j in range(2):
            # Integración numérica de cada término
            term1 = sum(w * (dN_lagrange(i)/h_e) * (dN_lagrange(j)/h_e) * h_e 
                       for xi, w in zip(xi_gauss, w_gauss))
            term2 = sum(w * N_lagrange(xi, i) * (dN_lagrange(j)/h_e) * h_e 
                       for xi, w in zip(xi_gauss, w_gauss))
            term3 = sum(w * N_lagrange(xi, i) * N_lagrange(xi, j) * h_e 
                       for xi, w in zip(xi_gauss, w_gauss))
            
            K_local[i, j] = D*term1 + U*term2 + k*term3
    
    return K_local

def assemble_global_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=10):
    """Ensamblaje de matriz global y aplicación de BCs."""
    n_nodes = len(nodes)
    n_elem = len(connectivity)
    h_e = nodes[1] - nodes[0]
    
    K_global = np.zeros((n_nodes, n_nodes))
    F_global = np.zeros(n_nodes)
    
    # Ensamblaje elemento por elemento
    for e in range(n_elem):
        node_ids = connectivity[e, :]
        K_local = Ke_local(h_e, D, U, k, quad_n=quad_n)
        for i_local in range(2):
            i_global = node_ids[i_local]
            for j_local in range(2):
                j_global = node_ids[j_local]
                K_global[i_global, j_global] += K_local[i_local, j_local]
    
    # Aplicación de condición Robin en frontera de entrada
    K_global[0, 0] += U
    F_global[0] = U * c_inlet
    
    return K_global, F_global

def solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=10):
    """Solución del sistema lineal resultante."""
    K, F = assemble_global_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=quad_n)
    return np.linalg.solve(K, F)

## Generación de figuras para el informe

In [ ]:
# Figura 1: Solución analítica del problema
x_plot = np.linspace(0, L, 500)
c_analitica = solucion_analitica(x_plot, D, U, k, L, c_inlet)

plt.figure(figsize=(10, 6))
plt.plot(x_plot, c_analitica, 'k-', lw=2.5)
plt.xlabel(r'$x$ [m]', fontsize=13)
plt.ylabel(r'$c$ [mol/L]', fontsize=13)
plt.title('Solución Analítica del Reactor PFR', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('figuras/Fig1_Solucion_Analitica.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"c(0) = {c_analitica[0]:.2f} mol/L")
print(f"c(L) = {c_analitica[-1]:.2f} mol/L")
print(f"Conversión = {(c_inlet - c_analitica[-1])/c_inlet*100:.1f}%")

In [ ]:
# Figura 2: Convergencia del método de Legendre para diferentes M
X = np.linspace(0, L, 400)
M_values = [3, 5, 8, 12, 15, 20]

plt.figure(figsize=(12, 7))
colors = plt.cm.viridis(np.linspace(0, 0.9, len(M_values)))

for idx, M in enumerate(M_values):
    K, F = assemble_KF_legendre(M, D, U, k, L, c_inlet, quad_n=120)
    a = np.linalg.solve(K, F)
    xi_points = 2 * X / L - 1
    c_approx = sum(a[j] * Nm(xi_points, j) for j in range(M))
    plt.plot(X, c_approx, '-', color=colors[idx], lw=2, label=f'M = {M}')

plt.plot(X, solucion_analitica(X, D, U, k, L, c_inlet), 'k--', lw=2.5, label='Analítica')
plt.xlabel(r'$x$ [m]', fontsize=13)
plt.ylabel(r'$c$ [mol/L]', fontsize=13)
plt.title('Método de Galerkin con Polinomios de Legendre', fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('figuras/Fig2_Legendre_Convergencia_M.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Figura 3: Análisis de convergencia - Error y residuo vs M
M_conv = np.arange(3, 30, 1)
errores_L2 = []
residuos = []

x_error = np.linspace(0, L, 500)
c_exact = solucion_analitica(x_error, D, U, k, L, c_inlet)

for M in M_conv:
    K, F = assemble_KF_legendre(M, D, U, k, L, c_inlet, quad_n=120)
    a = np.linalg.solve(K, F)
    xi_error = 2 * x_error / L - 1
    
    c_approx = np.zeros_like(x_error)
    dc_dx = np.zeros_like(x_error)
    d2c_dx2 = np.zeros_like(x_error)
    
    for m in range(M):
        Pm = Legendre.basis(m)
        c_approx += a[m] * Pm(xi_error)
        dc_dx += a[m] * Pm.deriv()(xi_error) * (2/L)
        d2c_dx2 += a[m] * Pm.deriv(2)(xi_error) * (2/L)**2
    
    error_L2 = np.sqrt(np.trapz((c_approx - c_exact)**2, x_error) / L)
    errores_L2.append(error_L2)
    
    R = D*d2c_dx2 - U*dc_dx - k*c_approx
    residuos.append(np.sqrt(np.mean(R**2)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogy(M_conv, errores_L2, 'bo-', lw=2, markersize=7)
ax1.set_xlabel(r'Número de funciones base $M$', fontsize=12)
ax1.set_ylabel(r'Error $L_2$', fontsize=12)
ax1.set_title('Convergencia del Error', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.semilogy(M_conv, residuos, 'ro-', lw=2, markersize=7)
ax2.set_xlabel(r'Número de funciones base $M$', fontsize=12)
ax2.set_ylabel(r'Norma RMS del Residuo', fontsize=12)
ax2.set_title('Convergencia del Residuo', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('figuras/Fig3_Convergencia_Error_Residuo.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Error L2 (M=3):  {errores_L2[0]:.2e}")
print(f"Error L2 (M=29): {errores_L2[-1]:.2e}")
print(f"Reducción: {errores_L2[0]/errores_L2[-1]:.1f}x")

In [ ]:
# Figura 4: Convergencia FEM para diferentes niveles de refinamiento
n_elem_list = [10, 20, 40, 60, 100, 150]

plt.figure(figsize=(12, 7))
colors_fem = plt.cm.coolwarm(np.linspace(0.1, 0.9, len(n_elem_list)))

for idx, n_elem in enumerate(n_elem_list):
    nodes, h, connectivity = create_mesh(L, n_elem)
    c_fem = solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=15)
    plt.plot(nodes, c_fem, 'o-', color=colors_fem[idx], lw=1.8, 
             markersize=4, label=f'{n_elem} elementos')

plt.plot(x_plot, c_analitica, 'k--', lw=2.5, label='Analítica')
plt.xlabel(r'$x$ [m]', fontsize=13)
plt.ylabel(r'$c$ [mol/L]', fontsize=13)
plt.title('Método de Elementos Finitos con Lagrange', fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('figuras/Fig4_FEM_Convergencia_Mallas.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Figura 5: Análisis de convergencia FEM - Error vs tamaño de malla
n_elem_conv = [5, 10, 20, 30, 40, 50, 60, 80, 100, 150, 200]
errores_L2_fem = []
h_values = []

for n_elem in n_elem_conv:
    nodes, h, connectivity = create_mesh(L, n_elem)
    c_fem = solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=15)
    c_fem_interp = np.interp(x_error, nodes, c_fem)
    error_L2 = np.sqrt(np.trapz((c_fem_interp - c_exact)**2, x_error) / L)
    errores_L2_fem.append(error_L2)
    h_values.append(h)

plt.figure(figsize=(10, 6))
plt.loglog(h_values, errores_L2_fem, 'bs-', lw=2, markersize=8, label='Error FEM')
h_ref = np.array(h_values)
plt.loglog(h_ref, 0.02*h_ref**2, 'k--', lw=2, alpha=0.6, label=r'$\mathcal{O}(h^2)$')
plt.xlabel('Tamaño de malla h [m]', fontsize=13)
plt.ylabel(r'Error $L_2$', fontsize=13)
plt.title('Convergencia FEM', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.savefig('figuras/Fig5_FEM_Convergencia_h.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Error L2 ({n_elem_conv[0]} elem):  {errores_L2_fem[0]:.2e}")
print(f"Error L2 ({n_elem_conv[-1]} elem): {errores_L2_fem[-1]:.2e}")
print(f"Reducción: {errores_L2_fem[0]/errores_L2_fem[-1]:.1f}x")

In [ ]:
# Figura 6: Comparación de ambos métodos numéricos
M_comp = 15
n_elem_comp = 100

K_leg, F_leg = assemble_KF_legendre(M_comp, D, U, k, L, c_inlet, quad_n=120)
a_leg = np.linalg.solve(K_leg, F_leg)
xi_comp = 2 * x_error / L - 1
c_legendre = sum(a_leg[j] * Nm(xi_comp, j) for j in range(M_comp))

nodes_fem, h_fem, conn_fem = create_mesh(L, n_elem_comp)
c_lagrange = solve_FEM(nodes_fem, conn_fem, D, U, k, c_inlet, quad_n=15)
c_lagrange_interp = np.interp(x_error, nodes_fem, c_lagrange)

plt.figure(figsize=(12, 7))
plt.plot(x_error, c_exact, 'k-', lw=3, label='Analítica', zorder=1)
plt.plot(x_error, c_legendre, 'b--', lw=2.5, label=f'Legendre (M={M_comp})', zorder=2)
plt.plot(x_error, c_lagrange_interp, 'r:', lw=2.5, label=f'FEM ({n_elem_comp} elem)', zorder=3)
plt.xlabel(r'$x$ [m]', fontsize=13)
plt.ylabel(r'$c$ [mol/L]', fontsize=13)
plt.title('Comparación de Métodos', fontsize=14, fontweight='bold')
plt.legend(fontsize=12, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('figuras/Fig6_Comparacion_Metodos.png', dpi=300, bbox_inches='tight')
plt.show()

error_leg = np.sqrt(np.trapz((c_legendre - c_exact)**2, x_error) / L)
error_fem = np.sqrt(np.trapz((c_lagrange_interp - c_exact)**2, x_error) / L)
print(f"\nError L2 Legendre: {error_leg:.2e}")
print(f"Error L2 FEM:      {error_fem:.2e}")

In [ ]:
# Figura 7: Estudio paramétrico - efecto de la tasa de reacción k
k_values = [k/4, k/2, k, 2*k, 4*k]

fig, axes = plt.subplots(2, 2, figsize=(15, 11))
colors_k = plt.cm.Spectral(np.linspace(0.1, 0.9, len(k_values)))

# Subplot 1: Perfiles de concentración
for idx, k_val in enumerate(k_values):
    c_k = solucion_analitica(x_error, D, U, k_val, L, c_inlet)
    axes[0, 0].plot(x_error, c_k, '-', color=colors_k[idx], lw=2.5, 
                    label=f'k = {k_val:.2f} 1/hr')
axes[0, 0].set_xlabel(r'$x$ [m]', fontsize=12)
axes[0, 0].set_ylabel(r'$c$ [mol/L]', fontsize=12)
axes[0, 0].set_title('Efecto de k en el Perfil de Concentración', fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Subplot 2: Perfiles normalizados
for idx, k_val in enumerate(k_values):
    c_k = solucion_analitica(x_error, D, U, k_val, L, c_inlet)
    axes[0, 1].plot(x_error/L, c_k/c_inlet, '-', color=colors_k[idx], lw=2.5, 
                    label=f'k = {k_val:.2f} 1/hr')
axes[0, 1].set_xlabel(r'Posición adimensional $x/L$', fontsize=12)
axes[0, 1].set_ylabel(r'$c/c_{inlet}$', fontsize=12)
axes[0, 1].set_title('Perfiles Normalizados', fontsize=13, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Subplot 3: Concentración de salida
k_range = np.linspace(0.1, 8, 100)
c_salida = [solucion_analitica(np.array([L]), D, U, kv, L, c_inlet)[0] for kv in k_range]
axes[1, 0].plot(k_range, c_salida, 'b-', lw=2.5)
axes[1, 0].axvline(k, color='r', linestyle='--', lw=2, alpha=0.7, label=f'k = {k} 1/hr')
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    axes[1, 0].plot(k_val, c_out, 'ro', markersize=9)
axes[1, 0].set_xlabel(r'Tasa de reacción $k$ [1/hr]', fontsize=12)
axes[1, 0].set_ylabel(r'$c(L)$ [mol/L]', fontsize=12)
axes[1, 0].set_title('Concentración de Salida vs k', fontsize=13, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Subplot 4: Conversión del reactor
conversion = [(c_inlet - c_out)/c_inlet * 100 for c_out in c_salida]
axes[1, 1].plot(k_range, conversion, 'g-', lw=2.5)
axes[1, 1].axvline(k, color='r', linestyle='--', lw=2, alpha=0.7, label=f'k = {k} 1/hr')
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    axes[1, 1].plot(k_val, (c_inlet - c_out)/c_inlet * 100, 'ro', markersize=9)
axes[1, 1].set_xlabel(r'Tasa de reacción $k$ [1/hr]', fontsize=12)
axes[1, 1].set_ylabel('Conversión [%]', fontsize=12)
axes[1, 1].set_title('Conversión del Reactor vs k', fontsize=13, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('figuras/Fig7_Estudio_Parametrico_k.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nEstudio Paramétrico de k:")
print(f"{'k [1/hr]':>10} {'c(L) [mol/L]':>15} {'Conversión [%]':>18}")
print("-" * 50)
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    conv = (c_inlet - c_out)/c_inlet * 100
    print(f"{k_val:10.2f} {c_out:15.2f} {conv:18.1f}")

In [ ]:
print("\n" + "="*70)
print("GENERACIÓN DE FIGURAS COMPLETADA")
print("="*70)
print("\nArchivos generados en carpeta 'figuras/' (300 dpi):")
print("  1. Fig1_Solucion_Analitica.png")
print("  2. Fig2_Legendre_Convergencia_M.png")
print("  3. Fig3_Convergencia_Error_Residuo.png")
print("  4. Fig4_FEM_Convergencia_Mallas.png")
print("  5. Fig5_FEM_Convergencia_h.png")
print("  6. Fig6_Comparacion_Metodos.png")
print("  7. Fig7_Estudio_Parametrico_k.png")
print("="*70)